## 시계열 예측기법을 활용한 상점별 매출예측

* 1. data down sampling
* 2. data scaling
* 3. ARIMA

In [1]:
import pandas as pd
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%precision 3

'%.3f'

In [2]:
train = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_상점매출/train.csv')
test = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_상점매출/test.csv')
submission = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_상점매출/submission.csv')

In [4]:
# date,time => python datetime 형식으로 변경
test['datetime'] = test['date'] + test['time']

# date,time col 합쳐서 datetime으로
test['datetime'] = pd.to_datetime(test['datetime'], format='%Y-%m-%d%H:%M:%S', errors='raise')

# datetime을 인덱스로
test.set_index('datetime',inplace=True)
del test['date']
del test['time']

In [42]:
test.head()

,store_id,card_id,amount,installments,days_of_week,holyday
datetime,,,,,,
2016-08-01 00:28:15,0,bf33518373,125,NaN,0,0
2016-08-01 01:09:58,0,7a19a3a92f,90,NaN,0,0
2016-08-01 01:47:24,0,6f9fd7e241,150,NaN,0,0
2016-08-01 17:54:43,0,8bcf1d61b2,362,NaN,0,0
2016-08-01 18:48:53,0,6a722ce674,125,NaN,0,0


In [62]:
# down sampling

# 마지막 거래일로부터 100일의 총 매출을 예측해야함
# 1일씩 예측은 위험성이 높으므로, 25일씩 예측
# 30 -> 20 -> 10 -> 7

a = test.query("store_id =='0'")['amount'].resample('M').sum()